### Estimación de indicadores de vivienda para el Plan General de Desarrollo de la Ciudad de México

En este script se descargan los microdatos de la Encuesta Nacional de Ingresos y Gastos de los Hogares (ENIGH) para los años 2016, 2018, 2020 y 2022. Posteriormente, se estiman dos indicadores de vivienda para la Ciudad de México: rezago habitacional y viviendas sin escrituras. Los resultados se presentan en dos cuadros.

In [1]:
##Estimación de indicadores de vivienda
#Se borra todo lo que se encuentra en el entorno

rm(list=ls())

# Librerías ====
if(!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, showtext, srvyr)

Loading required package: pacman



In [2]:
#Crear directorio para guardar los archivos
dir.create("data", showWarnings = FALSE)

In [3]:
url_basica<-"https://www.inegi.org.mx/contenidos/programas/enigh/nc/"
segunda_parte<-"microdatos/enigh"
fin<-"_ns_viviendas_csv.zip"

In [4]:
years<-c("2016", "2018", "2020", "2022")

In [5]:
# Descargar los archivos, descomprimirlos y renombrarlos
for (i in 1:length(years)) {
  url <- paste0(url_basica, years[i], "/", segunda_parte, years[i], fin)
  destfile <- paste0("data/", years[i], fin)
  
  # Descargar el archivo
  download.file(url, destfile = destfile)
  
  # Descomprimir el archivo
  unzip(destfile, exdir = "data")
  
  # Renombrar el archivo descomprimido a "viviendas_xxxx.csv"
  file.rename(
    from = "data/viviendas.csv", 
    to = paste0("data/viviendas_", years[i], ".csv")
  )
}    

### Función de estimación

In [6]:
estimacion <- function(path, variable) {
  # Cargar los datos
  datos <- read_csv(path)
  
  datos <- datos %>%
    mutate(
      across(starts_with("mat") & where(is.character),
             ~ replace_na(parse_number(.x, na = c('', 'NA', '&')), 0)),
      rezago = if_else(
        ((tot_resid / num_cuarto) > 2.5) |
          (mat_pared %in% 1:6) |
          (mat_techos %in% c(1:4, 6, 7, 9)) |
          (mat_pisos == 1) |
          (excusado == 2),
        "En rezago",
        "Fuera de rezago"
      )
    ) %>%
    mutate(cve_ent = case_when(
      nchar(folioviv) == 9  ~ paste0("0", substr(folioviv, 1, 1)),
      nchar(folioviv) == 10 ~ substr(folioviv, 1, 2)
    )) %>%
    # Filtrar si entidad es 9
  filter(cve_ent == "09")
  
  # Definir diseño muestral
  mydesign <- datos %>%
    as_survey_design(ids = upm,
                     strata = est_dis,
                     weights = factor)
  
  # Estimar rezago habitacional
  rezago <- mydesign %>%
    group_by(rezago) %>%
    summarise(
      viviendas = survey_total(vartype = "cv"),
      pct = survey_prop(vartype = "cv")
    ) %>%
    mutate(pct = round(pct * 100, 2)) %>%
    filter(rezago == "En rezago") %>%
    # Añadir año
    mutate(year = substr(path, 16, 19))
  
  
  
  # Estimar viviendas sin escrituras
  escrituras <- mydesign %>%
    filter(tenencia %in% c(3,4)) %>%
group_by(escrituras) %>%
    summarise(
      viviendas = survey_total(vartype = "cv"),
      pct = survey_prop(vartype = "cv")
    ) %>%
    filter(escrituras == 3) %>%
    mutate(pct = round(pct * 100, 2),
           escrituras = case_when(
        escrituras == 3 ~ "Sin escrituras"
))%>%
    
mutate(year = substr(path, 16, 19))
  
  # Devolver el resultado basado en la variable
  if (variable == "rezago") {
    return(rezago)
  } else if (variable == "sin_escrituras") {
    return(escrituras)
  } else {
    stop("Variable no reconocida. Use 'rezago' o 'sin_escrituras'.")
  }
}


In [7]:
#Construir cuadros de rezago habitacional

tabla_rezago<-map_dfr(
  list.files("data", full.names = TRUE, pattern = "viviendas_.*.csv"),
  estimacion,
  variable = "rezago"
) 

tabla_rezago


Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 69169 Columns: 64
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (10): folioviv, tipo_viv, mat_techos, tipo_finan, num_dueno1, num_dueno2...
dbl (54): mat_pared, mat_pisos, antiguedad, antigua_ne, cocina, cocina_dor, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 73405 Columns: 64
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (9): folioviv, tipo_viv, mat_techos, tipo_finan, num_dueno1, num_dueno2...
dbl (55): mat_pared, mat_pisos, antiguedad, antigua_ne, cocina, cocina_dor, ...

ℹ Use 

rezago,viviendas,viviendas_cv,pct,pct_cv,year
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
En rezago,272200,0.08809435,9.81,0.08684886,2016
En rezago,218501,0.08906203,8.02,0.08727815,2018
En rezago,198021,0.08360376,7.31,0.08292165,2020
En rezago,175242,0.09933936,5.94,0.09870452,2022


In [9]:
#Construir cuadros de viviendas sin escrituras
#Viviendas sin escrituras corresponden a viviendas propias y propias que se están pagando y no cuentan con escrituras

tabla_escrituras<-map_dfr(
    list.files("data", full.names = TRUE, pattern = "viviendas_.*.csv"),
    estimacion,
    variable = "sin_escrituras"
    ) 
    
tabla_escrituras

Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 69169 Columns: 64
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (10): folioviv, tipo_viv, mat_techos, tipo_finan, num_dueno1, num_dueno2...
dbl (54): mat_pared, mat_pisos, antiguedad, antigua_ne, cocina, cocina_dor, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 73405 Columns: 64
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (9): folioviv, tipo_viv, mat_techos, tipo_finan, num_dueno1, num_dueno2...
dbl (55): mat_pared, mat_pisos, antiguedad, antigua_ne, cocina, cocina_dor, ...

ℹ Use 

escrituras,viviendas,viviendas_cv,pct,pct_cv,year
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
Sin escrituras,266547,0.08646642,17.71,0.08298775,2016
Sin escrituras,246283,0.09985847,17.22,0.09461881,2018
Sin escrituras,259056,0.09464920,16.23,0.08765464,2020
Sin escrituras,310191,0.08428046,17.63,0.07835935,2022
